In [ ]:
import os
import sys
from langchain_openai import OpenAI

from dotenv import load_dotenv
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
print(api_key)

### File System

Giving access to writing files ( + to add reading files from root directory, python, json, txt, pdfs, cvs)

[https://python.langchain.com/docs/integrations/tools/filesystem]

### CHROMA RAG

In [7]:
# Load
# from langchain_community.document_loaders import WebBaseLoader
# loader = WebBaseLoader("https://python.langchain.com/docs/integrations/retrievers/self_query/chroma_self_query")
# data = loader.load()

from langchain_community.document_loaders import PyPDFLoader
# Load PDF document
loader = PyPDFLoader("../BukGPT/data/CharlesB2.pdf")
# pages = loader.load_and_split()
data = loader.load()

# LLM
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Add to vectorDB
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-private",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


# RAG chain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke("What does the document say about humanity?"))

The document suggests that humanity, at its best, can exhibit gentleness, understanding, courage, and acts of kindness. However, overall, humanity is portrayed as a mass or glob lacking in positive qualities. It describes humanity as being like a large animal deep in sleep, difficult to awaken, and when activated, it tends towards brutality, selfishness, unjust judgments, and murder. The document also highlights the treachery, hatred, violence, and absurdity present in the average human being, suggesting that even those who preach against negative behaviors may be the best at committing them. Overall, the document paints a bleak picture of humanity's nature.


### FAISS Similarity docs retrival

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Load PDF document
loader = PyPDFLoader("../BukGPT/data/CharlesB2.pdf")
pages = loader.load_and_split()

embeddings = OpenAIEmbeddings()
faiss_index = FAISS.from_documents(pages, embeddings)

# Define the chatbot function
def RAG_similarity(question):
    # Query the vector database
    docs = faiss_index.similarity_search(question, k=5)

    # Print the most relevant document
    print("Relevant document:")
    for doc in docs:
        print(str(doc.metadata["page"]) + ":", doc.page_content[:200])

# Test the chatbot
RAG_similarity("technology")

Relevant document:
79: www.PoemHunter.com - The World's Poetry Archive
80 
my computer
"what?" they say, "you got a
<i>computer</i>?"
it's like I have sold out to
the enemy.
I had no idea so many
people were prejudiced
agai
1: www.PoemHunter.com - The World's Poetry Archive
2 
16-bit Intel 8088 chip
with an Apple Macintosh
you can't run Radio Shack programs
in its disc drive.
nor can a Commodore 64
drive read a file
you hav
123: www.PoemHunter.com - The World's Poetry Archive
124 
their bodies while their bodies are still
alive enough to transmit and feel and run up
and down without locks and paychecks and
ideals and possessi
80: www.PoemHunter.com - The World's Poetry Archive
81 
come flying
out,
better than
ever.
I have no
idea what causes
all this
computer
prejudice.
me?
I want to go
the next step
beyond the
computer.
I'm s
92: www.PoemHunter.com - The World's Poetry Archive
93 
I don't know).
they were born with money and
they don't have to dirty their hands in
slaughterhouses or w

### Memory

### Ollama Embeddings 

In [15]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama2")

text = "This is a test document."

query_result = embeddings.embed_query(text)
query_result[:5]

[-0.1023407056927681,
 0.012098119594156742,
 0.18036691844463348,
 0.16426609456539154,
 0.21228839457035065]

In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama

model_name = "gemma"
llm = Ollama(
    model=model_name,
    system="""
    You are always pragmatic and word efficient in your answers. 
    Write in the shortest way posssible, using the fewest words while preserving the essence of the message. 
    Always respond in markdown format.""",
    # template="",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    temperature=1.0,
    # stop=["10. "],
    # verbose=True,
)

question = "is this real life?"

print(f"Question: {question}")

response = llm.invoke(question)


Question: is this real life?
Real life? Not virtual. This is a digital conversation, not a physical one. I am a language model, not a human.